# Vane calibration (see also GBT memo 302)

The VANE/SKY calibration in ARGUS ...

The CALSEQ calibration in the W-band receiver ...


In [ ]:
# @todo clean up some imports not needed anymore
import os
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
from scipy.stats import norm
from astropy.io import fits
import astropy.units as u

from dysh.fits.sdfitsload import SDFITSLoad
from dysh.fits.gbtfitsload import GBTFITSLoad
from dysh.util.files import dysh_data
from dysh.util.selection import Selection
from dysh.spectra.core import mean_tsys
# new
from dysh.fits.core import mean_data
from dysh.fits.core import getbeam
from dysh.fits.core import calseq
from dysh.fits.core import vanecal
from dysh.fits.core import plot_vegas

In [ ]:
# not needed anymore
k=['DATE-OBS','SCAN', 'IFNUM', 'PLNUM', 'FDNUM', 'INTNUM', 'PROCSCAN','FEED', 'SRFEED', 'FEEDXOFF', 'FEEDEOFF', 'SIG', 'CAL', 'PROCSEQN', 'PROCSIZE']
ks=['DATE-OBS','SCAN', 'SUBOBSMODE', 'IFNUM', 'PLNUM', 'FDNUM', 'INTNUM', 'CAL', 'PROCSEQN']
kw=['DATE-OBS','SCAN', 'SUBOBSMODE', 'IFNUM', 'PLNUM', 'FDNUM', 'INTNUM', 'CAL', 'PROCSEQN', 'CALPOSITION','TCAL','TSYS']


Helper functions

In [ ]:
def mkdir(name, clean=True):
    """ simpler frontend for making a directory that might also already exist
        clean=True:    also remove files inside
    """
    os.makedirs(name, exist_ok = True)
    if clean:
        fns = os.listdir(name)
        for fn in fns:
            print(f"Removing {fn} from {name}")
            os.unlink(os.path.join(name,fn))

In [ ]:
# stop gap until spf.getnod() is smart again
def getnod(sdf, scans, beams, ifnum=0, plnum=0, tsys=None):
    """ fake getnod() based on alternating gettp() with averaging done internally
        use the real sdf.getnod() for final analysis
        new API:    scans[], feeds[], tsys[]
        """
    if tsys is None:
        tsys = [1.0, 1.0]
    else:
        print(f"@todo use tsys={tsys} for proper weighting")
    ps1_on = sdf.gettp(scan=scans[0], fdnum=beams[0], ifnum=ifnum, plnum=plnum, calibrate=True, cal=False).timeaverage()
    ps1_off = sdf.gettp(scan=scans[1], fdnum=beams[0], ifnum=ifnum, plnum=plnum, calibrate=True, cal=False).timeaverage()
    sp1 = (ps1_on - ps1_off)/ps1_off

    ps2_on = sdf.gettp(scan=scans[1], fdnum=beams[1], ifnum=ifnum, plnum=plnum, calibrate=True, cal=False).timeaverage()
    ps2_off = sdf.gettp(scan=scans[0], fdnum=beams[1], ifnum=ifnum, plnum=plnum, calibrate=True, cal=False).timeaverage()
    sp2 = (ps2_on - ps2_off)/ps2_off

    return (sp1,sp2)


##   NGC0001

no obvious signal.

In [ ]:
f1 = dysh_data('AGBT21B_024_01/AGBT21B_024_01.raw.vegas')
sdf1 = GBTFITSLoad(f1, skipflags=True)

In [ ]:
sdf1.summary()

# NGC5908

This edge2 takes too long, but has signal. Only need scans 327..334

In [ ]:
f2=dysh_data('AGBT21B_024_14/AGBT21B_024_14.raw.vegas')
print(f2)
f2 = "edge2"
sdf2 = GBTFITSLoad(f2)

In [ ]:
sdf2.summary()

In [ ]:
plot_vegas(sdf2,[327,329],"edge2 TP at 112 and 114 GHz")

In [ ]:
plot_vegas(sdf2,[327,329],"edge2 Tsys at 112 and 114 GHz",tsys=True, ylim=[0.5,1.2])

In [ ]:
beam2 = getbeam(sdf2)   # 1,9
print("feeds",beam2)


In [ ]:
tsys1 = vanecal(sdf2, [327, 328], feeds=beam2)
print(tsys1)
tsys2 = vanecal(sdf2, [329, 330], feeds=beam2)
print(tsys2)

In [ ]:
print(tsys1)
print(tsys2)

In [ ]:
sp1,sp2 = getnod(sdf2, [331, 332], beam2, tsys=tsys1)
sp3,sp4 = getnod(sdf2, [333, 334], beam2, tsys=tsys2)
sp3 = sp1.average([sp2,sp3,sp4])
object = sp3.meta['OBJECT']

In [ ]:
sp3.plot(title=f"Source: {object}",xaxis_unit="km/s")

In [ ]:
# do a baseline subtraction
kms = u.km/u.s
sp4=sp3[400:600]    # @todo need to figure this out in km/s
sp4.baseline(model="poly", degree=5, exclude=[-150*kms,150*kms], remove=True)
#sp4.baseline(model="cheby", degree=5, exclude=[-150*kms,150*kms], remove=True)
# chebyshev', 'legendre', or 'hermite'
print(sp4.baseline_model)
sp4.plot(title=f"Source: {object}",xaxis_unit="km/s")

sp5 = sp4.smooth('box', 3)
sp5.plot(title=f"Source: {object}",xaxis_unit="km/s")


#  M82 Q-band (CALSEQ)

4 different IFs. Has signal in all but the first IF, but since IF's are overlapping, more interesting to plot them in one
spectrum.   We don't have tools for that, so there's a cheat in the end.

In [ ]:
f3 = dysh_data(accept='AGBT15B_244_07/AGBT15B_244_07.raw.vegas')
print(f3)
sdf3=GBTFITSLoad(f3, skipflags=True)

In [ ]:
sdf3.summary()

In [ ]:
beams3 = getbeam(sdf3)    
print(beams3)

In [ ]:
tsys3,g = calseq(sdf3, 130)     # 100.27992034259859, 9.129371938341321e-07
print(tsys3,g)

In [ ]:
print(tsys3,g)  # 100.27992034259859, 9.129371938341321e-07

The 2nd IF (`ifnum=1`) has a line at 87.3 GHz, though there's also a line for ifnum=2 (88.6), and two lines for ifnum=3 (88.6 and 89.1 GHz), obviously the different IFs overlap somewhat. The edges are noisy and we'll have to take off.

In [ ]:
sp_final = list(range(4))

for ifnum in range(4):
    sp = []
    for s in range(131,140,2):
        print(s)
        sp1,sp2 = getnod(sdf3, [s,s+1], [0,1], plnum=0, ifnum=ifnum)
        sp3,sp4 = getnod(sdf3, [s,s+1], [0,1], plnum=1, ifnum=ifnum)
        sp.append(sp1)
        sp.append(sp2)
        sp.append(sp3)
        sp.append(sp4)
    
    sp_final[ifnum] = sp[0].average(sp[1:]).smooth('gauss',16)
    sp_final[ifnum].plot()

The edges are unreliable, so we take off 50 channels from the ends. These spectra have 1024 channels. Scale by Tsys.

In [ ]:
edge = 50
x = list(range(4))
y = list(range(4))
for ifnum in range(4):
    sp = sp_final[ifnum][edge:-edge] * tsys3
    object = sp.meta['OBJECT']
    sp.plot(xaxis_unit="GHz", title=f"{object}, ifnum={ifnum}")
    # sp.plot(xaxis_unit="km/s", title=f"{object}, ifnum={ifnum}")
    #    cheat and save the numerical values of the spectra
    x[ifnum] = sp.frequency.value
    y[ifnum] = sp.flux.value


Now a cheat, the 4 IFs were stored in arrays x,y

In [ ]:
for i in range(4):
    plt.plot(x[i],y[i])
plt.xlabel("Frequency (GHz)")
plt.ylabel("Ta")
plt.title(f"{object}")

* 88.63185 HCN
* 89.18853 HCO+

* VHEL = 269 km/s   dFreq ~ 0.08 GHz